In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-11-17 21:41:18.952526


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Albums From Known People

In [7]:
data = getFile("/Volumes/Music/MusicDB/toget.p")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

_ = clock("Last Run")

In [ ]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
toget={}
known={}

for ia,artistName in enumerate(data):
    #artistID = artistName
    artistID = None
    if artistID is None:
        retval = getMusicData('Name', artistName)
        if retval is not None:
            artistID = retval.index.values[0]
    modValue  = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
    if toget.get(modValue) is None:
        toget[modValue] = {}
    toget[modValue][artistID] = True
        
    if ia % 100 == 0:
        print(ia,'/',len(data))

In [ ]:
for modVal in toget.keys():
    artistIDs = toget[modVal]
    print(modVal,'\t',len(artistIDs))
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID in artistIDs:
        print(' ',artistID)
        artistData = dbdata[artistID]
        albs.downloadAlbumFromArtistData(artistID=artistID, artistData=artistData, mediaTypes=None)

# Albums From All Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=10, mediaTypes=None, rand=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists
Loading /Volumes/Music/Discog/artists-db/80-DB.p... Found 5675 Artists
Loading data from /Volumes/Music/Discog/diagnostic/albumKnownIDs.p
  --> This file is 1.0MB.
Loading /Volumes/Music/Discog/diagnostic/albumKnownIDs.p
<class 'artist.artistDataClass'>
<class 'artist.artistMediaClass'>
<class 'dict'>
1 / 4    	 1000180 	 Cercueil
  Found album code 1789672 with 1 artists
  Saving

  Found album code 3187247 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1002680/3187247.p
  --> This file is 16.5kB.
  Done and sleeping...
  Found album code 1351967 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1002680/1351967.p
  --> This file is 16.7kB.
  Done and sleeping...
  Found album code 2913645 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1002680/2913645.p
  --> This file is 17.2kB.
  Done and sleeping...
  Found album code 4339971 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1002680/4339971.p
  --> This file is 16.6kB.
  Done and sleeping...
  Found album code 1339546 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1002680/1339546.p
  --> This file is 17.7kB.
  Done and sleeping...
  Found album code 4386871 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1002680/4386871.p
  --> This file is 16.6kB.
  Done and sleeping...
  Found album code 4386881 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1002680/438

  Found album code 6621438 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1003480/6621438.p
  --> This file is 14.5kB.
  Done and sleeping...
<class 'artist.artistDataClass'>
<class 'artist.artistMediaClass'>
<class 'dict'>
0 / 3    	 1004480 	 Daria
  Found album code 1628862 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1004480/1628862.p
  --> This file is 17.0kB.
  Done and sleeping...
  Found album code 285737 with 2 artists
  Saving /Volumes/Music/Discog/albums/80/1004480/285737.p
  --> This file is 16.4kB.
  Saving /Volumes/Music/Discog/albums/26/814826/285737.p
  --> This file is 16.4kB.
  Done and sleeping...
  Found album code 2404204 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1004480/2404204.p
  --> This file is 16.1kB.
  Done and sleeping...
<class 'artist.artistDataClass'>
<class 'artist.artistMediaClass'>
<class 'dict'>
0 / 1    	 100580 	 Sporty & The Laptop
  Found album code 130393 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1

  Found album code 2251103 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1013180/2251103.p
  --> This file is 15.5kB.
  Done and sleeping...
  Found album code 832896 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1013180/832896.p
  --> This file is 15.3kB.
  Done and sleeping...
  Found album code 1160448 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1013180/1160448.p
  --> This file is 14.9kB.
  Done and sleeping...
  Found album code 2649825 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1013180/2649825.p
  --> This file is 14.6kB.
  Done and sleeping...
  Found album code 11233279 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1013180/11233279.p
  --> This file is 14.3kB.
  Done and sleeping...
  Found album code 5497877 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1013180/5497877.p
  --> This file is 14.5kB.
  Done and sleeping...
<class 'artist.artistDataClass'>
<class 'artist.artistMediaClass'>
<class 'dict'>
0 / 16    	 

  Found album code 7217106 with 1 artists
  Exists /Volumes/Music/Discog/albums/14/4497014/7217106.p
  Done and sleeping...
  Found album code 1197730 with 1 artists
  Saving /Volumes/Music/Discog/albums/NAN/NAN/1197730.p
  --> This file is 17.1kB.
  Done and sleeping...
  Found album code 2567227 with 1 artists
  Exists /Volumes/Music/Discog/albums/NAN/NAN/2567227.p
  Done and sleeping...
  --> This file is 1.0MB.

== Download Rate: 125 / 5.3 = 23.4

<class 'artist.artistDataClass'>
<class 'artist.artistMediaClass'>
<class 'dict'>
1 / 5    	 1016380 	 Knaughty Knights
  Found album code 371227 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1016380/371227.p
  --> This file is 16.5kB.
  Done and sleeping...
  Found album code 2016015 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1016380/2016015.p
  --> This file is 17.1kB.
  Done and sleeping...
  Found album code 374206 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1016380/374206.p
  --> This file is 17.5kB

...

  Found album code 7305063 with 1 artists
  Saving /Volumes/Music/Discog/albums/80/1025680/7305063.p
  --> This file is 14.4kB.
  Done and sleeping...
<class 'artist.artistDataClass'>
<class 'artist.artistMediaClass'>
<class 'dict'>
14 / 22    	 1027180 	 Carl Mathieu Lange
  Found album code 8804209 with 3 artists
  Saving /Volumes/Music/Discog/albums/89/855889/8804209.p
  --> This file is 15.8kB.
  Saving /Volumes/Music/Discog/albums/80/1027180/8804209.p
  --> This file is 15.8kB.
  Saving /Volumes/Music/Discog/albums/48/941848/8804209.p
  --> This file is 15.8kB.
  Done and sleeping...
  Found album code 4231946 with 3 artists
  Saving /Volumes/Music/Discog/albums/39/826839/4231946.p
  --> This file is 16.5kB.
  Saving /Volumes/Music/Discog/albums/40/826840/4231946.p
  --> This file is 16.5kB.
  Saving /Volumes/Music/Discog/albums/80/1027180/4231946.p
  --> This file is 16.5kB.
  Done and sleeping...
<class 'artist.artistDataClass'>
<class 'artist.artistMediaClass'>
<class 'dict'>
1

  Found album code 8036574 with 1 artists
  Saving /Volumes/Music/Discog/albums/45/1181345/8036574.p
  --> This file is 18.0kB.
  Done and sleeping...
  Found album code 6510668 with 1 artists
  Saving /Volumes/Music/Discog/albums/52/2484152/6510668.p
  --> This file is 17.9kB.
  Done and sleeping...
  Found album code 7618185 with 1 artists
  Saving /Volumes/Music/Discog/albums/45/257745/7618185.p
  --> This file is 18.0kB.
  Done and sleeping...
  Found album code 4815942 with 1 artists
  Exists /Volumes/Music/Discog/albums/55/253855/4815942.p
  Done and sleeping...
  Found album code 4816289 with 1 artists
  Exists /Volumes/Music/Discog/albums/55/253855/4816289.p
  Done and sleeping...
  Found album code 4872000 with 1 artists
  Saving /Volumes/Music/Discog/albums/20/57620/4872000.p
  --> This file is 20.4kB.
  Done and sleeping...
  Found album code 4817970 with 1 artists
  Saving /Volumes/Music/Discog/albums/55/253855/4817970.p
  --> This file is 21.0kB.
  Done and sleeping...
  F

  Found album code 11253104 with 2 artists
  Saving /Volumes/Music/Discog/albums/83/1149283/11253104.p
  --> This file is 16.0kB.
  Saving /Volumes/Music/Discog/albums/80/1031380/11253104.p
  --> This file is 16.0kB.
  Done and sleeping...
  Found album code 1308389 with 3 artists
  Saving /Volumes/Music/Discog/albums/72/180072/1308389.p
  --> This file is 16.4kB.
  Saving /Volumes/Music/Discog/albums/91/1030991/1308389.p
  --> This file is 16.4kB.
  Saving /Volumes/Music/Discog/albums/80/1031380/1308389.p
  --> This file is 16.4kB.
  Done and sleeping...
  Found album code 1547775 with 2 artists
  Saving /Volumes/Music/Discog/albums/80/1031380/1547775.p
  --> This file is 16.3kB.
  Saving /Volumes/Music/Discog/albums/5/621205/1547775.p
  --> This file is 16.3kB.
  Done and sleeping...
  Found album code 13526177 with 2 artists
  Saving /Volumes/Music/Discog/albums/83/1149283/13526177.p
  --> This file is 16.2kB.
  Saving /Volumes/Music/Discog/albums/80/1031380/13526177.p
  --> This fi

  Found album code 981045 with 2 artists
  Saving /Volumes/Music/Discog/albums/80/1033980/981045.p
  --> This file is 17.4kB.
  Saving /Volumes/Music/Discog/albums/98/1152498/981045.p
  --> This file is 17.4kB.
  Done and sleeping...
  Found album code 13927737 with 2 artists
  Saving /Volumes/Music/Discog/albums/80/1033980/13927737.p
  --> This file is 17.7kB.
  Saving /Volumes/Music/Discog/albums/98/1152498/13927737.p
  --> This file is 17.7kB.
  Done and sleeping...
  Found album code 924634 with 2 artists
  Saving /Volumes/Music/Discog/albums/80/1033980/924634.p
  --> This file is 15.5kB.
  Saving /Volumes/Music/Discog/albums/98/1152498/924634.p
  --> This file is 15.5kB.
  Done and sleeping...
  Found album code 1413389 with 4 artists
  Saving /Volumes/Music/Discog/albums/80/1033980/1413389.p
  --> This file is 17.0kB.
  Saving /Volumes/Music/Discog/albums/55/3317755/1413389.p
  --> This file is 17.0kB.
  Saving /Volumes/Music/Discog/albums/59/2414759/1413389.p
  --> This file is 

  Found album code 2789589 with 3 artists
  Exists /Volumes/Music/Discog/albums/12/1669012/2789589.p
  Saving /Volumes/Music/Discog/albums/60/1758460/2789589.p
  --> This file is 16.5kB.
  Exists /Volumes/Music/Discog/albums/27/835527/2789589.p
  Done and sleeping...
  Found album code 3647498 with 2 artists
  Saving /Volumes/Music/Discog/albums/NAN/NAN/3647498.p
  --> This file is 16.7kB.
  Saving /Volumes/Music/Discog/albums/71/1669471/3647498.p
  --> This file is 16.7kB.
  Done and sleeping...
  Found album code 1114962 with 2 artists
  Exists /Volumes/Music/Discog/albums/27/835527/1114962.p
  Exists /Volumes/Music/Discog/albums/28/835528/1114962.p
  Done and sleeping...
  Found album code 3008616 with 2 artists
  Exists /Volumes/Music/Discog/albums/27/835527/3008616.p
  Exists /Volumes/Music/Discog/albums/28/835528/3008616.p
  Done and sleeping...
  Found album code 7685543 with 2 artists
  Saving /Volumes/Music/Discog/albums/NAN/NAN/7685543.p
  --> This file is 17.6kB.
  Saving /V